## Mongodb Aggregate and group
1. avg
2. sum
3. project


In [2]:
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db=client['students']
studentscores = db['studentscores'] #we can use db.studentscores also to create collection

In [4]:
data = [
    {"user":"Ben", "subject":"Database", "score":80}, 
    {"user":"Stark",  "subject":"JavaScript", "score":90}, 
    {"user":"Stark",  "title":"Database", "score":85}, 
    {"user":"Ben",  "title":"JavaScript", "score":75}, 
    {"user":"Stark",  "title":"Data Science", "score":60},
    {"user":"Ben",  "title":"Data Science", "score":95}
]
studentscores.insert_many(data)

InsertManyResult([ObjectId('65fb0852d85d012824fdfa89'), ObjectId('65fb0852d85d012824fdfa8a'), ObjectId('65fb0852d85d012824fdfa8b'), ObjectId('65fb0852d85d012824fdfa8c'), ObjectId('65fb0852d85d012824fdfa8d'), ObjectId('65fb0852d85d012824fdfa8e')], acknowledged=True)

In [15]:
#find total no of subjects
result = studentscores.aggregate([{
    '$group': {'_id':'$user',              #using $user because its a field not string
    'total subjects':{'$sum':1}} # for every user incrementing sum by 1, total subjects is the new group name
    #[] is for using multiple operations, its for pipelining
}])

for i in result:
    print(i)

{'_id': 'Stark', 'total subjects': 3}
{'_id': 'Ben', 'total subjects': 3}


In [20]:
#calculate the total score of users
score_result = studentscores.aggregate([{
    '$group': {'_id': '$user',
               'Total score': {'$sum':'$score'}}
}])
for i in score_result:
    print(i)

{'_id': 'Stark', 'Total score': 235}
{'_id': 'Ben', 'Total score': 250}


In [22]:
#calulate the average based on user
average = studentscores.aggregate([{
    '$group': {'_id':'$user',
               'Average score': {'$avg': '$score'}}
}])

for i in average:
    print(i)

{'_id': 'Stark', 'Average score': 78.33333333333333}
{'_id': 'Ben', 'Average score': 83.33333333333333}


In [29]:
#creating new collection
import datetime
data = [{"_id" : 1, "item" : "abc", "price" : 10, "quantity" : 2, "date" : datetime.datetime.now(datetime.UTC)},
{ "_id" : 2, "item" : "jkl", "price" : 20, "quantity" : 1, "date" : datetime.datetime.now(datetime.UTC) },
{ "_id" : 3, "item" : "xyz", "price" : 5, "quantity" : 5, "date" : datetime.datetime.now(datetime.UTC) },
{ "_id" : 4, "item" : "abc", "price" : 10, "quantity" : 10, "date" : datetime.datetime.now(datetime.UTC) },
{ "_id" : 5, "item" : "xyz", "price" : 5, "quantity" : 10, "date" :datetime.datetime.now(datetime.UTC)}]


In [35]:
storecollection = db.stores
storecollection.insert_many(data)

InsertManyResult([1, 2, 3, 4, 5], acknowledged=True)

In [48]:
agg_result=storecollection.aggregate([
   {
      "$group": {
          "_id": '$item',
         "avgAmount": {"$avg": {"$multiply": [ "$price", "$quantity" ]}},
          "avgQuantity": { "$avg": "$quantity" }
      }
   }
])

for i in agg_result:
    print(i)

{'_id': 'abc', 'avgAmount': 60.0, 'avgQuantity': 6.0}
{'_id': 'jkl', 'avgAmount': 20.0, 'avgQuantity': 1.0}
{'_id': 'xyz', 'avgAmount': 37.5, 'avgQuantity': 7.5}


In [50]:
## $Project
##its like < select column1, column2 from tablename >
##used to select fields

data=[{
  "_id" : 1,
  "title": "abc123",
  "isbn": "0001122223334",
  "author": { "last": "zzz", "first": "aaa" },
  "copies": 5
},
{
  "_id" : 2,
  "title": "Baked Goods",
  "isbn": "9999999999999",
  "author": { "last": "xyz", "first": "abc", "middle": "" },
  "copies": 2
}
]

In [51]:
bookscollection = db['books']

In [52]:
bookscollection.insert_many(data)

InsertManyResult([1, 2], acknowledged=True)

In [58]:
result = bookscollection.aggregate([
    {
        '$project':
        {'title':1,'isbn':1}
    }
])

for i in result: 
    print(i)

{'_id': 1, 'title': 'abc123', 'isbn': '0001122223334'}
{'_id': 2, 'title': 'Baked Goods', 'isbn': '9999999999999'}
